# Segmenting and Clustering Neighborhoods in Toronto

# Part 1. Data Scraping

TODOS:

* The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

* Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

* More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.

* If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.

Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.


In [1]:
import pandas as pd

## 1. Data extraction

We will extract tables from this [Wikipedia Page](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M 'List_of_postal_codes_of_Canada').

This pages contains list of postal codes of Canada where the first letter is M. Postal codes beginning with M are located within the city of Toronto in the province of Ontario. 
Let's take a look at first few rows and columns of tables scraped from that URL and identify our targeted table.

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
tables = pd.read_html(url)
print(f'\nRead HTML tables in Wikipedia URL : \n{"▋"*80}')
# Number of tables in tables
print(f'\nNumber of tables: {len(tables)}\n')
# Find targeted table by printing few rows and columns
for i, table in enumerate(tables): 
    print(f'First row of Table {i}:')
    print(f'{table.iloc[:1,:3]}')
    print()


Read HTML tables in Wikipedia URL : 
▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋

Number of tables: 3

First row of Table 0:
                 0                1                         2
0  M1ANot assigned  M2ANot assigned  M3ANorth York(Parkwoods)

First row of Table 1:
     0                      1    2
0  NaN  Canadian postal codes  NaN

First row of Table 2:
    0   1   2
0  NL  NS  PE



## 2. Data cleansing

The Table 0 contains our targeted data. Each cell of this table contain the following information:

1. Postal code: the first three characters are postal code, corresponding to the Forward Sortation Area.

2. Borough: Name of borough. Some postal code have not been assigned to any borough. In such case, it is coded as 'Not assigned' 

3. Neighborhood: Names of neighborhood is inside parentheses (i.e. round brackets), append to name of borough

### Create the DataFrame `df`
First, we will create a DataFrame and split each cell of Table 0 into 3 part: PostalCode, Borough, and Neighborhood and remove rows that contains postal code that is not assigned to any borough/neighborhood.

In [3]:
df = tables[0].to_numpy().transpose().flatten() # select table with index 0 and flatten to a np array
df = pd.DataFrame(df, columns=['data']).data.str.extract(r'(.{3})(.*?)\((.*)')
df.columns = ['PostalCode', 'Borough', 'Neighborhood']
df.dropna(subset = ['Borough'] ,inplace = True) # Remove 'Not Asigned' row
df.reset_index(drop = True, inplace = True)
df.head()


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge)
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek)
2,M1E,Scarborough,Guildwood / Morningside / West Hill)
3,M1G,Scarborough,Woburn)
4,M1H,Scarborough,Cedarbrae)


### Let's verify that postal code is in correct format: 

The postal code in Toronto must begin with letter 'M', following by a number and one leter from A to Z.

In [4]:
# Verify that Postal Code is in correct format
sum(1-df.PostalCode.str.match(r'M\d[A-Z]')) # return number of mismatch

0

### Let's see the unique values of Borough name in column `Borough` and recode if necessary

In [5]:
# Borough columns
df.Borough.unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'East YorkEast Toronto', 'Central Toronto', 'Downtown Toronto',
       'Downtown TorontoStn A PO Boxes25 The Esplanade', 'York',
       'West Toronto', "Queen's Park",
       'MississaugaCanada Post Gateway Processing Centre',
       'East TorontoBusiness reply mail Processing Centre969 Eastern',
       'Etobicoke', 'EtobicokeNorthwest'], dtype=object)

In [6]:
df['Borough'] = df['Borough'].replace({
    'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
    'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
    'EtobicokeNorthwest':'Etobicoke Northwest',
    'East YorkEast Toronto':'East York/East Toronto',
    'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.Borough.unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'East York/East Toronto', 'Central Toronto', 'Downtown Toronto',
       'Downtown Toronto Stn A', 'York', 'West Toronto', "Queen's Park",
       'Mississauga', 'East Toronto Business', 'Etobicoke',
       'Etobicoke Northwest'], dtype=object)

### Recode column `Neighborhood`

In [7]:
df.Neighborhood.unique()

array(['Malvern / Rouge)', 'Rouge Hill / Port Union / Highland Creek)',
       'Guildwood / Morningside / West Hill)', 'Woburn)', 'Cedarbrae)',
       'Scarborough Village)',
       'Kennedy Park / Ionview / East Birchmount Park)',
       'Golden Mile / Clairlea / Oakridge)',
       'Cliffside / Cliffcrest / Scarborough Village West)',
       'Birch Cliff / Cliffside West)',
       'Dorset Park / Wexford Heights / Scarborough Town Centre)',
       'Wexford / Maryvale)', 'Agincourt)',
       "Clarks Corners / Tam O'Shanter / Sullivan)",
       "Milliken / Agincourt North / Steeles East / L'Amoreaux East)",
       "Steeles West / L'Amoreaux West)", 'Upper Rouge)',
       'Hillcrest Village)', 'Fairview / Henry Farm / Oriole)',
       'Bayview Village)', 'York Mills / Silver Hills)',
       'Willowdale / Newtonbrook)', 'Willowdale)South',
       'York Mills West)', 'Willowdale)West', 'Parkwoods)',
       'Don Mills)North', 'Don Mills)South(Flemingdon Park)',
       'Bathurst Manor / Wilso

In [8]:
df['Neighborhood']= df['Neighborhood'].str.replace(' /',',').str.replace(')',' ')
df['Neighborhood']= df['Neighborhood'].str.replace('(', ', ').str.strip(' ')
df.Neighborhood.unique()

array(['Malvern, Rouge', 'Rouge Hill, Port Union, Highland Creek',
       'Guildwood, Morningside, West Hill', 'Woburn', 'Cedarbrae',
       'Scarborough Village',
       'Kennedy Park, Ionview, East Birchmount Park',
       'Golden Mile, Clairlea, Oakridge',
       'Cliffside, Cliffcrest, Scarborough Village West',
       'Birch Cliff, Cliffside West',
       'Dorset Park, Wexford Heights, Scarborough Town Centre',
       'Wexford, Maryvale', 'Agincourt',
       "Clarks Corners, Tam O'Shanter, Sullivan",
       "Milliken, Agincourt North, Steeles East, L'Amoreaux East",
       "Steeles West, L'Amoreaux West", 'Upper Rouge',
       'Hillcrest Village', 'Fairview, Henry Farm, Oriole',
       'Bayview Village', 'York Mills, Silver Hills',
       'Willowdale, Newtonbrook', 'Willowdale South', 'York Mills West',
       'Willowdale West', 'Parkwoods', 'Don Mills North',
       'Don Mills South, Flemingdon Park',
       'Bathurst Manor, Wilson Heights, Downsview North',
       'Northwood Par

### Final result of data cleansing process:

In [9]:
print('First few rows of the DataFrame df:')
print(df.head())

First few rows of the DataFrame df:
  PostalCode      Borough                            Neighborhood
0        M1B  Scarborough                          Malvern, Rouge
1        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek
2        M1E  Scarborough       Guildwood, Morningside, West Hill
3        M1G  Scarborough                                  Woburn
4        M1H  Scarborough                               Cedarbrae


In [10]:
print(f'Shape of df: {df.shape}')

Shape of df: (103, 3)
